**Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients.**
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem. ##

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

>**Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

>**Doctor**: What was his interarm BP? _(blood pressure)_

>**Nurse**: Systolic was 140 on the right; 110 on the left.

>**Doctor**: It's an aortic dissection! Get to the OR _(operating room)_ now!

>_(intense music playing)_

In this fictitious  scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data. ##

**1. Read in the data.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score
from sklearn.preprocessing import  StandardScaler
from sklearn.linear_model import LogisticRegression
#PolynomialFeatures,

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv('./chronic_kidney_disease_full.csv')

In [3]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [4]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

**2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?**

**Answer:**

Age, blood pressure, red blood cell count, sodium, potassium, white blood cell count, etc., and whether or not they exhibit symptoms of/have CKD.

---

## Step 3: Explore the data. ##

**3. How much of the data is missing from each column?**

In [5]:
df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [6]:
df_copy = df.dropna()

In [7]:
df_copy.shape

(158, 25)

**4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?**

**Answer:**


158 rows would remain.

One downside of dropped the rows with missing values are that losing the data in the columns that do have valid values.

Another downside of dropped the rows are that dealing with a 'random' sample of respondents.



**5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.**

**Answer:**

> 1. Any features that making by human standard can be human error.
> 2. Some columns deal with largely data, consume a lot of time.
> 3. Some records may have been mistakenly adjusted/rounded/reported incorrectly.

---

## Step 4: Model the data. ##

**6. Suppose that I want to construct a model where no person who has chronic kidney disease (CKD) will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?**

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

**Answer:**

Build a model that tells everyone that they have CKD; the only error that could occur is a `False Positive`.

**7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?**

**Answer:**

Optimized Sensitivity and `the False Negative` Rate by minimizing  `False Negatives`.

**8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?**

**Answer:**


People who don't have CKD (False Positives)will be undue stress and worry.

**9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?**

**Answer:**

Build a model that tells everyone that they do not have CKD; the only error that could occur is a Type II Error, or a False Negative.

**10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?**

**Answer:**

Optimized Specificity and the False Positive Rate by minimizing  False Positives.

**11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?**

**Answer:**

People who have CKD (False Negatives),They will not take any action to help themselves.

**12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.**

> Hint: Remember to do a train/test split!

In [8]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   rbc     248 non-null    object 
 6   pc      335 non-null    object 
 7   pcc     396 non-null    object 
 8   ba      396 non-null    object 
 9   bgr     356 non-null    float64
 10  bu      381 non-null    float64
 11  sc      383 non-null    float64
 12  sod     313 non-null    float64
 13  pot     312 non-null    float64
 14  hemo    348 non-null    float64
 15  pcv     329 non-null    float64
 16  wbcc    294 non-null    float64
 17  rbcc    269 non-null    float64
 18  htn     398 non-null    object 
 19  dm      398 non-null    object 
 20  cad     398 non-null    object 
 21  appet   399 non-null    object 
 22  pe

In [10]:
df.drop(['rbc','pc','sod','pot','pcv','wbcc','rbcc'], axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     391 non-null    float64
 1   bp      388 non-null    float64
 2   sg      353 non-null    float64
 3   al      354 non-null    float64
 4   su      351 non-null    float64
 5   pcc     396 non-null    object 
 6   ba      396 non-null    object 
 7   bgr     356 non-null    float64
 8   bu      381 non-null    float64
 9   sc      383 non-null    float64
 10  hemo    348 non-null    float64
 11  htn     398 non-null    object 
 12  dm      398 non-null    object 
 13  cad     398 non-null    object 
 14  appet   399 non-null    object 
 15  pe      399 non-null    object 
 16  ane     399 non-null    object 
 17  class   400 non-null    object 
dtypes: float64(9), object(9)
memory usage: 56.4+ KB


---

## Step 5: Evaluate the model. ##

**13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.**

In [12]:
df_version_1 = df

In [13]:
df_version_1.replace(np.nan, 0, inplace=True)

In [14]:
df_version_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     400 non-null    float64
 1   bp      400 non-null    float64
 2   sg      400 non-null    float64
 3   al      400 non-null    float64
 4   su      400 non-null    float64
 5   pcc     400 non-null    object 
 6   ba      400 non-null    object 
 7   bgr     400 non-null    float64
 8   bu      400 non-null    float64
 9   sc      400 non-null    float64
 10  hemo    400 non-null    float64
 11  htn     400 non-null    object 
 12  dm      400 non-null    object 
 13  cad     400 non-null    object 
 14  appet   400 non-null    object 
 15  pe      400 non-null    object 
 16  ane     400 non-null    object 
 17  class   400 non-null    object 
dtypes: float64(9), object(9)
memory usage: 56.4+ KB


In [15]:
df_version_1['pcc'].value_counts()

notpresent    354
present        42
0               4
Name: pcc, dtype: int64

In [16]:
df_version_1['pcc'].replace(0, 'present', inplace=True)

In [17]:
df_version_1['pcc'].value_counts()

notpresent    354
present        46
Name: pcc, dtype: int64

In [18]:
df_version_1['ba'].value_counts()

notpresent    374
present        22
0               4
Name: ba, dtype: int64

In [19]:
df_version_1['ba'].replace(0, 'present', inplace=True)

In [20]:
df_version_1['ba'].value_counts()

notpresent    374
present        26
Name: ba, dtype: int64

In [21]:
df_version_1['htn'].value_counts()

no     251
yes    147
0        2
Name: htn, dtype: int64

In [22]:
df_version_1['htn'].replace(0, 'yes', inplace=True)

In [23]:
df_version_1['dm'].value_counts()

no     261
yes    137
0        2
Name: dm, dtype: int64

In [24]:
df_version_1['dm'].replace(0, 'yes', inplace=True)

In [25]:
df_version_1['dm'].value_counts()

no     261
yes    139
Name: dm, dtype: int64

In [26]:
df_version_1['cad'].value_counts()

no     364
yes     34
0        2
Name: cad, dtype: int64

In [27]:
df_version_1['cad'].replace(0, 'yes', inplace=True)

In [28]:
df_version_1['cad'].value_counts()

no     364
yes     36
Name: cad, dtype: int64

In [29]:
df_version_1['appet'].value_counts()

good    317
poor     82
0         1
Name: appet, dtype: int64

In [30]:
df_version_1['appet'].replace(0, 'poor', inplace=True)

In [31]:
df_version_1['appet'].value_counts()

good    317
poor     83
Name: appet, dtype: int64

In [32]:
df_version_1['pe'].value_counts()

no     323
yes     76
0        1
Name: pe, dtype: int64

In [33]:
df_version_1['pe'].replace(0, 'yes', inplace=True)

In [34]:
df_version_1['pe'].value_counts()

no     323
yes     77
Name: pe, dtype: int64

In [35]:
df_version_1['ane'].value_counts()

no     339
yes     60
0        1
Name: ane, dtype: int64

In [36]:
df_version_1['ane'].replace(0, 'yes', inplace=True)

In [37]:
df_version_1['ane'].value_counts()

no     339
yes     61
Name: ane, dtype: int64

In [38]:
df_version_1['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

In [39]:
df_version_1 = pd.get_dummies(data=df_version_1, drop_first=True)

In [40]:
df_version_1.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes', 'class_notckd'],
      dtype='object')

In [41]:
df_version_1.drop('class_notckd', axis=1, inplace=True)

In [42]:
df_version_1['class'] = df['class']

In [43]:
df_version_1.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'hemo', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes', 'class'],
      dtype='object')

In [44]:
df_version_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          400 non-null    float64
 1   bp           400 non-null    float64
 2   sg           400 non-null    float64
 3   al           400 non-null    float64
 4   su           400 non-null    float64
 5   bgr          400 non-null    float64
 6   bu           400 non-null    float64
 7   sc           400 non-null    float64
 8   hemo         400 non-null    float64
 9   pcc_present  400 non-null    uint8  
 10  ba_present   400 non-null    uint8  
 11  htn_yes      400 non-null    uint8  
 12  dm_yes       400 non-null    uint8  
 13  cad_yes      400 non-null    uint8  
 14  appet_poor   400 non-null    uint8  
 15  pe_yes       400 non-null    uint8  
 16  ane_yes      400 non-null    uint8  
 17  class        400 non-null    object 
dtypes: float64(9), object(1), uint8(8)
memory usage: 3

In [45]:
df_version_1.head()

,age,bp,sg,al,su,bgr,bu,sc,hemo,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_poor,pe_yes,ane_yes,class
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,15.4,0,0,1,1,0,0,0,0,ckd
1,7.0,50.0,1.020,4.0,0.0,0.0,18.0,0.8,11.3,0,0,0,0,0,0,0,0,ckd
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,9.6,0,0,0,1,0,1,0,1,ckd
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,11.2,1,0,1,0,0,1,1,1,ckd
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,11.6,0,0,0,0,0,0,0,0,ckd


In [46]:
# df_version_1.drop('ane_yes', axis=1, inplace=True)

In [47]:
# df_version_1.drop('pe_yes', axis=1, inplace=True)

In [48]:
# df_version_1.drop('cad_yes', axis=1, inplace=True)

In [49]:
# df_version_1.drop('dm_yes', axis=1, inplace=True)

In [50]:
# df_version_1.drop('htn_yes', axis=1, inplace=True)

In [51]:
# df_version_1.drop('ba_present', axis=1, inplace=True)

In [52]:
# df_version_1.drop('pcc_present', axis=1, inplace=True)

In [53]:
# df_version_1.drop('bu', axis=1, inplace=True)

In [54]:
version1_columns_list = list(df_version_1)

In [55]:
version1_features = []

In [56]:
[version1_features.append(col) for col in version1_columns_list if col != 'class']

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [57]:
version1_features

['age',
 'bp',
 'sg',
 'al',
 'su',
 'bgr',
 'bu',
 'sc',
 'hemo',
 'pcc_present',
 'ba_present',
 'htn_yes',
 'dm_yes',
 'cad_yes',
 'appet_poor',
 'pe_yes',
 'ane_yes']

In [96]:
X = df_version_1[version1_features]
y = df_version_1['class']

In [97]:
X

,age,bp,sg,al,su,bgr,bu,sc,hemo,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_poor,pe_yes,ane_yes
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,15.4,0,0,1,1,0,0,0,0
1,7.0,50.0,1.020,4.0,0.0,0.0,18.0,0.8,11.3,0,0,0,0,0,0,0,0
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,9.6,0,0,0,1,0,1,0,1
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,11.2,1,0,1,0,0,1,1,1
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,11.6,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,140.0,49.0,0.5,15.7,0,0,0,0,0,0,0,0
396,42.0,70.0,1.025,0.0,0.0,75.0,31.0,1.2,16.5,0,0,0,0,0,0,0,0
397,12.0,80.0,1.020,0.0,0.0,100.0,26.0,0.6,15.8,0,0,0,0,0,0,0,0
398,17.0,60.0,1.025,0.0,0.0,114.0,50.0,1.0,14.2,0,0,0,0,0,0,0,0


In [98]:
y.value_counts(normalize=True)

ckd       0.625
notckd    0.375
Name: class, dtype: float64

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [100]:
ss = StandardScaler()

In [101]:
ss.fit(X_train)

StandardScaler()

In [102]:
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [103]:
logreg = LogisticRegression()

In [104]:
logreg.fit(X_train_sc, y_train)

LogisticRegression()

In [105]:
logreg.score(X_train_sc, y_train)

0.96

In [106]:
logreg.score(X_test_sc, y_test)

0.89

**14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.**

In [107]:
logreg.coef_

array([[ 0.20476081, -0.44112612,  0.00668784, -2.74605357, -0.68290591,
        -0.11102797,  0.58862124, -1.39695511,  1.39030167, -0.10253848,
         0.56514608, -1.36090092, -1.67072225, -0.4892265 , -0.59060027,
        -0.0266757 , -0.56297009]])

In [108]:
np.exp(0.11018577)

1.1164854606988666

**15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.**

**Answer:**
>`simple` always easy, and can `predict probabilities`.

**Advantages of Logistic Regression:**

    1.  Logistic Regression will predict continuous values.
    2.  Logistic Regression connects regression and classification.
    3.  Logistic Regression can predict probabilities.

**16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.**

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

In [109]:
predictions = logreg.predict(X_test_sc)

In [110]:
from sklearn.metrics import confusion_matrix, classification_report

In [111]:
cm = confusion_matrix(y_test, predictions)

In [112]:
cm

array([[57,  6],
       [ 5, 32]])

In [113]:
cm = pd.DataFrame(cm, columns=['Predicted Negative','Predicted Positive'], index=['Actual Negative','Actual Positive'])

In [114]:
cm

,Predicted Negative,Predicted Positive
Actual Negative,57,6
Actual Positive,5,32


In [115]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ckd       0.92      0.90      0.91        63
      notckd       0.84      0.86      0.85        37

    accuracy                           0.89       100
   macro avg       0.88      0.88      0.88       100
weighted avg       0.89      0.89      0.89       100



**17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)**

**Answer:**

In this hospital case, I want to optimize for`sensitivity`; although predicting someone is positive for CKD when they are actually negative, a False Negative, is a bad mistake to make, it is not as bad as predicting someone is negative for CKD when they are actually positve, a False Positive.

A False Positive, on the other hand, would result in a sick person going to do with their life as if they were healthy, possibly leading to even more serious health concerns or even death.



**18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.**

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

In [116]:
from matplotlib import pyplot as plt
import seaborn as sns

**19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?**

**Answer:**

Unbalanced classes are generally a problem because the minority class is at risk of not having enough exposure during the model process to be accounted for in the model. 

I would take action by offering to build two versions of the model, one that does not make any changes to offset the unbalanced classes, and one that sets both classes equal by either: 1.  `negative CKD` cases; or 2. `positive CKD` cases.  This would allow us to observe if there are any measurable differences in the models' results.


In [117]:
df['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

In [118]:
df_version_1['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

**20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?**

**21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.**


1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [119]:
df_version_1.shape

(400, 18)

In [120]:
df_version_1['class'].value_counts()

ckd       250
notckd    150
Name: class, dtype: int64

**22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?**
> Be sure to look at how well it performs on non-CKD data.

In [121]:

X = df_version_1[['age','bp','sg','al','su','bgr']]
y = df_version_1['class']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=y)

In [123]:
ss.fit(X_train)

StandardScaler()

In [124]:
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [125]:
logreg.fit(X_train_sc, y_train)

LogisticRegression()

In [126]:
logreg.score(X_train_sc, y_train)

0.8733333333333333

In [127]:
logreg.score(X_test_sc, y_test)

0.88

In [128]:
predictions = logreg.predict(X_test_sc)

In [129]:
cm = confusion_matrix(y_test, predictions)

In [130]:
cm

array([[51, 12],
       [ 0, 37]])

In [131]:
cm = pd.DataFrame(cm, columns=['Predicted Negative','Predicted Positive'], index=['Actual Negative','Actual Positive'])

In [132]:
cm

,Predicted Negative,Predicted Positive
Actual Negative,51,12
Actual Positive,0,37


In [134]:
51+0+12+37

100

**In this case, as there are 0 False Negative out of a total of 100 predictions.  Generally, unbalanced classes allow models enough exposure to the minority class to be able to recognize a minority instance in predictions but for 12 of False Positives  we can retest again.**



## Step 6: Answer the problem. ##

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

**Improving the model performance by choosing a secondary model that specifically potential targets.**